#CLUSTERING : DATASET KENDARAAN
#oleh:	Muhammad Shulhannur	mshulhannur@gmail.com


#LAKUKAN IMPORT LIBRARIES

In [ ]:
#untuk data analysis and manipulation
import pandas as pd

#untuk matematika
import numpy as np

#untuk matematika
import math

#untuk generate random number
import random

#untuk grafik 2D
import matplotlib.pyplot as plt

#untuk visualisasi data ke grafik 2D berbasis matplotlib
import seaborn as sns

#untuk url
import io

#untuk request file from url
import requests

#DATA PREPROCESSING

##READ FILES INTO DATASETS

In [ ]:
# Read kendaraan_train.csv file into DataFrame
url_datatraining="https://cdn.discordapp.com/attachments/756550576640360469/833533583331164190/kendaraan_train.csv"
training=requests.get(url_datatraining).content
datatraining = pd.read_csv(io.StringIO(training.decode('utf-8')))
datatraining

In [ ]:
# Read kendaraan_test.csv file into DataFrame
url_datatesting="https://cdn.discordapp.com/attachments/756550576640360469/833533558941024266/kendaraan_test.csv"
testing=requests.get(url_datatesting).content
datatesting = pd.read_csv(io.StringIO(testing.decode('utf-8')))
datatesting

##LAKUKAN PENGECEKKAN MISSING DATA


In [ ]:
#LAKUKAN PENGECEKKAN MISSING DATA TRAINING
missing_datatraining = datatraining.isnull()
missing_datatraining

In [ ]:
for column in missing_datatraining.columns.values.tolist():
  print(column)
  print(missing_datatraining[column].value_counts())
  print("")

In [ ]:
datatraining.info()

In [ ]:
#LAKUKAN PENGECEKKAN MISSING DATA TESTING
missing_datatesting = datatesting.isnull()
missing_datatesting

In [ ]:
for column in missing_datatesting.columns.values.tolist():
  print(column)
  print(missing_datatesting[column].value_counts())
  print("")

In [ ]:
datatesting.info()

##DROP UNUSED/UNECESARY COLUMN

In [ ]:
datatraining.drop('id', axis=1, inplace=True)
datatraining.drop('Tertarik', axis=1, inplace=True)

datatraining

In [ ]:
datatesting.drop('Tertarik', axis=1, inplace=True)

datatesting

##MERGE DATA TESTING DAN DATA TRAINING

In [ ]:
DataKendaraan = datatraining.append(datatesting, ignore_index=True)

DataKendaraan

###LAKUKAN EKSPERIMEN DATA

In [ ]:
#PILIH DUA OBYEK YANG SEKIRANYA PALING BER KORELASI
kanalPenjualan = DataKendaraan['Kanal_Penjualan']
premi = DataKendaraan['Premi']

dataNoClean = pd.DataFrame({
    'Kanal_Penjualan' : kanalPenjualan,
    'Premi' : premi
})

dataNoClean

##HANDLE MISSING VALUE

In [ ]:
# UMUR & JENIS KELAMIN => MODUS
# 24 & PRIA
DataKendaraan['Umur'].mode()
DataKendaraan['Jenis_Kelamin'].mode()

In [ ]:
DataKendaraan["Umur"].replace(np.nan, 24, inplace=True)
DataKendaraan['Jenis_Kelamin'].replace(np.nan, "Pria", inplace=True)

DataKendaraan.head()

In [ ]:
# UMUR KENDARAAN, SIM, SUDAH ASURANSI, KENDARAAN RUSAK => MODUS
DataKendaraan['Umur_Kendaraan'].mode()

DataKendaraan['Umur_Kendaraan'].replace(np.nan, "1-2 Tahun", inplace=True)

In [ ]:
# UMUR KENDARAAN, SIM, SUDAH ASURANSI => MODUS
DataKendaraan['SIM'].mode()

DataKendaraan['SIM'].replace(np.nan, 1.0, inplace=True)

In [ ]:
# UMUR KENDARAAN, SIM, SUDAH ASURANSI => MODUS
DataKendaraan['Sudah_Asuransi'].mode()

DataKendaraan['Sudah_Asuransi'].replace(np.nan, 0.0, inplace=True)

In [ ]:
# UMUR KENDARAAN, SIM, SUDAH ASURANSI => MODUS
DataKendaraan['Kendaraan_Rusak'].mode()

DataKendaraan['Kendaraan_Rusak'].replace(np.nan, "Pernah", inplace=True)

In [ ]:
# UMUR KENDARAAN, SIM, SUDAH ASURANSI => MODUS
DataKendaraan['Kode_Daerah'].mode()

DataKendaraan['Kode_Daerah'].replace(np.nan, 28.0, inplace=True)

In [ ]:
# PREMI KANAL_PENJUALAN LAMA_BERLANGGANAN => MEAN

avg_premi = DataKendaraan['Premi'].astype('float').mean(axis=0)
avg_Kanal = DataKendaraan['Kanal_Penjualan'].astype('float').mean(axis=0)
avg_Berlangganan = DataKendaraan['Lama_Berlangganan'].astype('float').mean(axis=0)

DataKendaraan['Premi'].replace(np.nan, avg_premi, inplace=True)
DataKendaraan['Kanal_Penjualan'].replace(np.nan, avg_Kanal, inplace=True)
DataKendaraan['Lama_Berlangganan'].replace(np.nan, avg_Berlangganan, inplace=True)

In [ ]:
missing_data = DataKendaraan.isnull()

In [ ]:
for column in missing_data.columns.values.tolist():
  print(column)
  print(missing_data[column].value_counts())
  print("")

##GANTI TIPE OBJEK MENJADI TIPE KATEGORI

In [ ]:
# UBAH OBJECT KE CATEGORY
object_column = DataKendaraan.select_dtypes(['object']).columns

category_column = DataKendaraan.select_dtypes(['category']).columns
DataKendaraan[object_column]= DataKendaraan[object_column].apply(lambda x: x.astype('category'))

In [ ]:
DataKendaraan.info()

###UBAH CATEGORICAL MENJADI NUMERICAL

In [ ]:
# CAT.CODES -> CATEGORY => NGUBAH DATA CATEGORY KE NUMERIC-1,2,3,...

DataKendaraan[object_column] = DataKendaraan[object_column].apply(lambda x: x.cat.codes)
DataKendaraan[category_column] = DataKendaraan[category_column].apply(lambda x: x.cat.codes)

In [ ]:
DataKendaraan.head()

##TAMPILKAN KOLOM YANG TERISI

In [ ]:
kanalPenjualan = DataKendaraan['Kanal_Penjualan']
premi = DataKendaraan['Premi']

dataCluster = pd.DataFrame({
    'Kanal_Penjualan' : kanalPenjualan,
    'Premi' : premi
})

dataWithOutlier = dataCluster
dataCluster

##LAKUKAN PENGECEKKAN PENCILAN

In [ ]:
def check_outlier(data):
  plt.figure(figsize=(60, 60))
  f, axes = plt.subplots(1, 2)
  sns.boxplot(y= data['Kanal_Penjualan'], ax= axes[0], color='yellow')
  sns.boxplot(y= data['Premi'], ax=axes[1], color='yellow')
  plt.subplots_adjust(wspace=1)

check_outlier(dataCluster)

##HANDLE OUTLIERS DENGAN DATA CLEANSING

In [ ]:
#handle outlier Premi
while True:
  qlo1, qlo3 = np.percentile(dataCluster['Premi'],[25,75])
  iqrlo = qlo3 - qlo1
  lowerlo = qlo1 - (1.5 * iqrlo)
  upperlo = qlo3 + (1.5 * iqrlo)
  outlierlo = dataCluster[(dataCluster['Premi'] < (lowerlo)) | (dataCluster['Premi'] > (upperlo))]
  print('amount of outlier data',outlierlo.shape[0]) #jumlah outlier data
  idxlo = outlierlo.index
  dataCluster.drop(idxlo, inplace=True) #drop outlier data
  if (outlierlo.shape[0] <= 0):
    break

dataCluster['Premi'].describe()

In [ ]:
def check_outlier_encode(data):
  plt.figure(figsize=(60, 60))
  f, axes = plt.subplots(1, 2)
  sns.boxplot(y=data['Kanal_Penjualan'], ax= axes[0], color='yellow')
  sns.boxplot(y= data['Premi'], ax=axes[1], color='yellow')
  plt.subplots_adjust(wspace=1)

check_outlier_encode(dataCluster)

dataCluster.to_csv(r'/content/data_ready.csv', index=False, header=True)

##NORMALISASIKAN TABEL

In [ ]:
from sklearn.preprocessing import StandardScaler

normalize = StandardScaler()
df = normalize.fit_transform(dataCluster)

df

#K-MEANS PROCESSES

##SHOW UNCLUSTERED PLOT

In [ ]:
plt.scatter(df[:,0],df[:,1],c='black',label='unclustered data')

plt.title('Unclustered Plots')

plt.show()

##BUAT K-MEANS FUNCTIONS

In [ ]:
def k_means(x,k, no_iterations):
    idx = np.random.choice(len(x), k)
    EuclidianDistance = np.array([]).reshape(x.shape[0],0)
    #PILIH CENTROID SECARA RANDOM 
    centroids = x[idx, :] 
     
    #MENCARI JARAK ANTARA TIAP CENTROID DAN SEMUA DATA POINT
    for i in range(k):
      tempDist = np.sum((x-centroids[i])**2,axis = 1)
      EuclidianDistance = np.c_[EuclidianDistance, tempDist]
    
    points = np.array([np.argmin(i) for i in EuclidianDistance])
     
    #MENGULANGI CARA DIATAS SEBANYAK ITERASI YANG DIINPUT
    for _ in range(no_iterations): 
        centroids = []
        EuclidianDistance = np.array([]).reshape(x.shape[0],0)
        for idx in range(k):
            #MELAKUKAN UPDATE CENTROID DENGAN MENGAMBIL RATA RATA CLUSTER
            temp_cent = x[points==idx].mean(axis=0) 
            centroids.append(temp_cent)
            
        centroids = np.vstack(centroids) #UPDATE CENTROID 
         
        for i in range(k):
          tempDist = np.sum((x-centroids[i])**2,axis = 1)
          EuclidianDistance = np.c_[EuclidianDistance, tempDist]
  
        points = np.array([np.argmin(i) for i in EuclidianDistance])
         
    return points, centroids

##BUAT ELBOW METHOD

In [ ]:
from sklearn.cluster import KMeans #HANYA UNTUK ELBOW METHOD
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k, max_iter=300)
    kmeanModel.fit(df)
    distortions.append(kmeanModel.inertia_)

plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

##LAKUKAN PLOTTING HASIL K-MEANS

In [ ]:
#DARI ELBOW METHOD SEBELUMNYA K TERBAIK ADALAH 4
label, centroid = k_means(df,4,300)
label_centre = ['centroid 1', 'centroid 2', 'centroid 3', 'centroid 4']
colors=['r', 'g', 'b', 'y', 'teal']
j = 0
 
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(df[label == i , 0] , df[label == i , 1] , c = colors[i], linewidths=1 ,alpha=0.5, edgecolors= 'k', label = i )

for centre in centroid:
    plt.scatter(centroid[: , 0] , centroid[: , 1] , s=200, c = 'violet',linewidths=1, edgecolors= 'k', label = label_centre[j])
    j += 1

plt.show()

##CARI NILAI SILHOUETTE METHOD TERBESAR DARI SETIAP N_CLUSTERS

In [ ]:
from sklearn.metrics import silhouette_score
sil_score = []

for n_cluster in range(3, 7):
    kmeans = KMeans(n_clusters=n_cluster).fit(df)
    label = kmeans.labels_
    sil_coeff = silhouette_score(df, label, metric='euclidean')
    sil_score.append(sil_coeff)
    print('Nilai Silhoutte Method untuk n_clusters = {} adalah {}'.format(n_cluster, sil_coeff))

#LAKUKAN EKSPERIMEN MENGGUNAKAN NILAI K YANG DIDAPAT

##K = 3, TETAPI DILAKUKAN DATA CLEANSING

In [ ]:
label, centroid = k_means(df,3,300)
label_centre = ['centroid 1', 'centroid 2', 'centroid 3', 'centroid 4']
colors=['r', 'g', 'b', 'y', 'teal']
j = 0
 
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(df[label == i , 0] , df[label == i , 1] , c = colors[i], linewidths=1 ,alpha=0.5, edgecolors= 'k', label = i )

for centre in centroid:
    plt.scatter(centroid[: , 0] , centroid[: , 1] , s=200, c = 'violet',linewidths=1, edgecolors= 'k', label = label_centre[j])
    j += 1

plt.show()

##K = 3

In [ ]:
from sklearn.preprocessing import StandardScaler

normalize = StandardScaler()
dataWithOutlier = normalize.fit_transform(dataWithOutlier)

In [ ]:
from sklearn.cluster import KMeans #HANYA UNTUK ELBOW METHOD
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k, max_iter=300)
    kmeanModel.fit(dataWithOutlier)
    distortions.append(kmeanModel.inertia_)

plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
label, centroid = k_means(dataWithOutlier,3,300)
label_centre = ['centroid 1', 'centroid 2', 'centroid 3', 'centroid 4']
colors=['r', 'g', 'b', 'y', 'teal']
j = 0
 
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(df[label == i , 0] , df[label == i , 1] , c = colors[i], linewidths=1 ,alpha=0.5, edgecolors= 'k', label = i )

for centre in centroid:
    plt.scatter(centroid[: , 0] , centroid[: , 1] , s=200, c = 'violet',linewidths=1, edgecolors= 'k', label = label_centre[j])
    j += 1

plt.show()

##K = 4

In [ ]:
label, centroid = k_means(dataWithOutlier,4,300)
label_centre = ['centroid 1', 'centroid 2', 'centroid 3', 'centroid 4']
colors=['r', 'g', 'b', 'y', 'teal']
j = 0
 
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(df[label == i , 0] , df[label == i , 1] , c = colors[i], linewidths=1 ,alpha=0.5, edgecolors= 'k', label = i )

for centre in centroid:
    plt.scatter(centroid[: , 0] , centroid[: , 1] , s=200, c = 'violet',linewidths=1, edgecolors= 'k', label = label_centre[j])
    j += 1

plt.show()